In [ ]:
import time
import numpy as np
import pandas as pd
import configparser
import tensorflow as tf
from sklearn import model_selection, utils

In [ ]:
from ipynb.fs.full.ImportExportData import exportDataframeToCSV,importCsvToDataframe
from ipynb.fs.full.EdfManipulation import read_and_store_data
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes
from ipynb.fs.full.FeatureClassificationMethods import *
from ipynb.fs.full.FeatureComputation import *
from ipynb.fs.full.FeatureNormalization import featureNormalization
from ipynb.fs.full.FeatureSelection import dimentionalityReduction
from ipynb.fs.full.DatasetBalancing import oversamplingSMOTE
from ipynb.fs.full.SplitDataset import createTrainingAndTestDatasets

#### Feature Extraction

In [ ]:
def LeftRightHemisphereFeatures(df, readEdf, sample_rate, step, pca_tolerance, smote_neighbors, csvExportFile):
    if readEdf:
        ndf = leftRightHemisphericChannels(df)
        features = pd.DataFrame(featureExtractionLeftRight (ndf, sample_rate, step))
        del ndf
        features = featureNormalization(features)
        reduced_features = dimentionalityReduction(features, pca_tolerance)
        del features
        smote_features = utils.shuffle(oversamplingSMOTE(reduced_features.drop('seizure', axis = 1), reduced_features['seizure'], smote_neighbors))
        smote_features.reset_index(drop = True, inplace = True)
        del reduced_features
        exportDataframeToCSV(smote_features, csvExportFile)
    else :
        smote_features = df
    return smote_features

##### Create Training and Testing Data

In [ ]:
def LeftRightHemisphereTrainTestData (smote_features, test_ratio, k_fold, perfInd):
    train_dat, test_dat, train_ind, test_ind = createTrainingAndTestDatasets(smote_features, test_ratio)
    results = pd.DataFrame(columns = perfInd)
    kf = model_selection.KFold(n_splits = k_fold, shuffle=True)
    return train_dat, test_dat, train_ind, test_ind, results, kf

#### SVM

In [ ]:
def LeftRightHemisphereSVM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd):
    print('Implementing SVM method...')
    start = time.time()
    SVM(train_dat, train_ind, test_dat, test_ind, results)
    SVM_Kfold(smote_features, kf, perfInd, results)
    end = time.time()
    print('SVM finished in', round(end - start,2), 'sec\n')

#### KNN

In [ ]:
def LeftRightHemisphereKNN(train_dat, test_dat, train_ind, test_ind, results, knn_neighbors, smote_features, kf, perfInd):
    print('Implementing KNN method...')
    start = time.time()
    KNN(train_dat, train_ind, test_dat, test_ind, knn_neighbors, results)
    KNN_Kfold(smote_features, knn_neighbors, kf, perfInd, results)
    end = time.time()
    print('KNN finished in', round(end - start,2),'sec\n')

#### Naive Bayes

In [ ]:
def LeftRightHemisphereNB(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd):
    print('Implementing Naive Bayes method...')
    start = time.time()
    NaiveBayes(train_dat, train_ind, test_dat, test_ind, results)
    NaiveBayes_Kfold(smote_features, kf, perfInd, results)
    end = time.time()
    print('Naive Bayes finished in', round(end - start,2),'sec\n')

#### Decision Trees

In [ ]:
def LeftRightHemisphereDT(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd):
    print('Implementing Decision Trees method...')
    start = time.time()
    DecisionTrees(train_dat, train_ind, test_dat, test_ind, results)
    DecisionTrees_Kfold(smote_features, kf, perfInd, results)
    end = time.time()
    print('Decision Trees finished in', round(end - start,2),'sec\n')

#### Random Forest

In [ ]:
def LeftRightHemisphereRF(train_dat, test_dat, train_ind, test_ind, results, rf_estimators, smote_features, kf, perfInd):
    print('Implementing Random Forest method...')
    start = time.time()
    RandomForest(train_dat, train_ind, test_dat, test_ind, rf_estimators, results)
    RandomForest_Kfold(smote_features, rf_estimators, kf, perfInd, results)
    end = time.time()
    print('Random Forest finished in', round(end - start,2),'sec\n')

#### LDA

In [ ]:
def LeftRightHemisphereLDA(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd):
    print('Implementing LDA method...')
    start = time.time()
    LDA(train_dat, train_ind, test_dat, test_ind, results)
    LDA_Kfold(smote_features, kf, perfInd, results)
    end = time.time()
    print('LDA finished in', round(end - start,2),'sec\n')

#### Logistic Regression

In [ ]:
def LeftRightHemisphereLR(train_dat, test_dat, train_ind, test_ind, results, max_iter, smote_features, kf, perfInd):
    print('Implementing Logistic Regression method...')
    start = time.time()
    LogisticRegression(train_dat, train_ind, test_dat, test_ind, results, max_iter)
    LogisticRegression_Kfold(smote_features, kf, perfInd, results, max_iter)
    end = time.time()
    print('Logistic Regression finished in', round(end - start,2),'sec\n')

#### LSTM

In [ ]:
def LeftRightHemisphereLSTM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd, lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function, batch):
    print('Implementing LSTM method...')
    X_train = np.reshape(train_dat.values, (train_dat.shape[0], 1, train_dat.shape[1]))
    y_train = train_ind.values.astype(int)
    X_test = np.reshape(test_dat.values, (test_dat.shape[0], 1, test_dat.shape[1]))
    y_test = test_ind.values.astype(int)

    print('Starting LSTM for Model 1...')
    start = time.time()
    lstm_model1 = Lstm1Model(lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function)
    LSTM_method (lstm_model1, batch, epochs, X_train, y_train, X_test, y_test, results, 'LSTM1')
    LSTM_method_Kfold (smote_features, kf, perfInd, lstm_model1, batch, epochs, results, 'LSTM1')
    end = time.time()
    print('LSTM for Model 1 finished in', round(end - start,2),'sec')
    
    print('Starting LSTM for Model 2...')
    start = time.time()
    lstm_model2 = Lstm2Model(lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function)
    LSTM_method (lstm_model2, batch, epochs, X_train, y_train, X_test, y_test, results, 'LSTM2')
    LSTM_method_Kfold (smote_features, kf, perfInd, lstm_model2, batch, epochs, results, 'LSTM2')
    end = time.time()
    print('LSTM for Model 2 finished in', round(end - start,2),'sec')

In [ ]:
def LeftRightHemispheresExperiment(df, channels, readEdf, csvImportFile, csvExportFile, dataset, sample_rate, time_window, test_ratio, pca_tolerance, smote_neighbors, k_fold, knn_neighbors, rf_estimators, max_iter, dropout, loss_function, epochs, lstm_units, dense1_units, dense2_units, batch, step):
    perfInd = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'MCC', 'Kappa']
    smote_features = LeftRightHemisphereFeatures(df, readEdf, sample_rate, step, pca_tolerance, smote_neighbors, csvExportFile)
    train_dat, test_dat, train_ind, test_ind, results, kf = LeftRightHemisphereTrainTestData (smote_features, test_ratio, k_fold, perfInd)
    LeftRightHemisphereSVM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    LeftRightHemisphereKNN(train_dat, test_dat, train_ind, test_ind, results, knn_neighbors, smote_features, kf, perfInd)
    LeftRightHemisphereNB(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    LeftRightHemisphereDT(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    LeftRightHemisphereRF(train_dat, test_dat, train_ind, test_ind, results, rf_estimators, smote_features, kf, perfInd)
    LeftRightHemisphereLDA(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    LeftRightHemisphereLR(train_dat, test_dat, train_ind, test_ind, results, max_iter, smote_features, kf, perfInd)
    LeftRightHemisphereLSTM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd, lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function, batch)
    return results